# Model generator

## Imports

In [1]:
# list of supported layers
from tensorflow.keras.layers import (
    Input,
    Activation,
    AveragePooling2D,
    BatchNormalization,
    Conv2D,
    Dense,
    Dropout,
    Flatten,
    GlobalAveragePooling2D,
    GlobalMaxPooling2D,
    Lambda, # only for polynomial activation in the form of `Lambda(lambda x: x**2+x)`
    MaxPooling2D,
    ReLU,
    Softmax,
    )
from tensorflow.keras import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import tf2onnx
import onnxruntime as ort

## Dataset generator

In [2]:
X_train = np.random.randint(0, 255, size=(50, 32, 32, 3))
X_test = np.random.randint(0, 255, size=(10, 32, 32, 3))

y_train = np.random.randint(0, 4, size=(50, 1))
y_test = np.random.randint(0, 4, size=(10, 1))

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [3]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((50, 32, 32, 3), (10, 32, 32, 3), (50, 4), (10, 4))

## Model definition

### Formulation

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=5, kernel_size=3, input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=30, kernel_size=2, ), 
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(8, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(12, activation='relu'),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.AveragePooling2D(),
    tf.keras.layers.MaxPooling2D(pool_size=(1, 1)),
    #tf.keras.layers.GlobalMaxPool2D(),
    #tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(name='layer_6'),
    tf.keras.layers.Dense(4, activation='softmax'),
])


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 32, 30, 5)         50        
                                                                 
 batch_normalization (BatchN  (None, 32, 30, 5)        20        
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 31, 29, 30)        630       
                                                                 
 batch_normalization_1 (Batc  (None, 31, 29, 30)       120       
 hNormalization)                                                 
                                                                 
 dense (Dense)               (None, 31, 29, 8)         248       
                                                                 
 dense_1 (Dense)             (None, 31, 29, 12)        1

### Training

In [6]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['acc']
    )

In [7]:
model.fit(X_train, y_train, epochs=15, batch_size=16,  validation_data=(X_test, y_test))

Epoch 1/15
4/4 [==============================] - 3s 176ms/step - loss: 1.6222 - acc: 0.2400 - val_loss: 1.3299 - val_acc: 0.5000
Epoch 2/15
4/4 [==============================] - 0s 19ms/step - loss: 1.4727 - acc: 0.3400 - val_loss: 2.2548 - val_acc: 0.1000
Epoch 3/15
4/4 [==============================] - 0s 19ms/step - loss: 1.4493 - acc: 0.3600 - val_loss: 2.7830 - val_acc: 0.1000
Epoch 4/15
4/4 [==============================] - 0s 19ms/step - loss: 1.4486 - acc: 0.3400 - val_loss: 2.5213 - val_acc: 0.1000
Epoch 5/15
4/4 [==============================] - 0s 18ms/step - loss: 1.4214 - acc: 0.3400 - val_loss: 1.9311 - val_acc: 0.1000
Epoch 6/15
4/4 [==============================] - 0s 18ms/step - loss: 1.3890 - acc: 0.3600 - val_loss: 1.5533 - val_acc: 0.1000
Epoch 7/15
4/4 [==============================] - 0s 17ms/step - loss: 1.3718 - acc: 0.4200 - val_loss: 1.4014 - val_acc: 0.4000
Epoch 8/15
4/4 [==============================] - 0s 19ms/step - loss: 1.3741 - acc: 0.4800 - va

## Save

### Tensorflow model - h5

In [8]:
# Save tensorflow model
model.save('model.h5')

In [9]:
for layer in model.layers:
    print(layer.__class__.__name__, layer.get_config())
    try:
        print(layer.get_config()['function'])
    except:
        pass
    print(layer.get_input_shape_at(0),layer.get_output_shape_at(0))
    try:
        print(layer.get_weights()[0].shape)
        print(layer.get_weights()[1].shape)
    except:
        pass

Conv1D {'name': 'conv1d', 'trainable': True, 'batch_input_shape': (None, 32, 32, 3), 'dtype': 'float32', 'filters': 5, 'kernel_size': (3,), 'strides': (1,), 'padding': 'valid', 'data_format': 'channels_last', 'dilation_rate': (1,), 'groups': 1, 'activation': 'linear', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
(None, 32, 32, 3) (None, 32, 30, 5)
(3, 3, 5)
(5,)
BatchNormalization {'name': 'batch_normalization', 'trainable': True, 'dtype': 'float32', 'axis': ListWrapper([3]), 'momentum': 0.99, 'epsilon': 0.001, 'center': True, 'scale': True, 'beta_initializer': {'class_name': 'Zeros', 'config': {}}, 'gamma_initializer': {'class_name': 'Ones', 'config': {}}, 'moving_mean_initializer': {'class_name': 'Zeros', 'config': {}}, 'moving_variance_in

### Onnx model - onnx

In [10]:
#Convert to Onnx
# Define the input signature for the conversion
input_signature = [tf.TensorSpec([None, 32, 32, 3], tf.float32, name="input")]

# Convert the model to ONNX
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature=input_signature, opset=13)

# Save the ONNX model to a file
with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## Input Json generator

In [11]:
import json


In [12]:
X = X_test[[0]]
y = model.predict(X)
y

1/1 [==============================] - 0s 291ms/step


array([[0.37830344, 0.3534843 , 0.12240438, 0.14580785]], dtype=float32)

In [13]:
with open("input.json", "w") as f:
    json.dump({'in': X.flatten().tolist()}, f)